# Creating a frequency finder auto-encoder

## Import Libraries

In [89]:
import tensorflow as tf
import numpy as np
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import callbacks
import tensorflow.keras as keras
from tensorflow.keras import layers
import umap
from babyplots import Babyplot
#import hdbscan
from matplotlib import pyplot
from models import *
from converters import *
from helpers import *
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

In [90]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get drive path 

In [54]:
drive_path = find_drive_path()
drive_path

'/media/mat2m10/Samsung_T5'

## Load Data from drive

In [8]:
data = pd.read_pickle(f"{drive_path}/Data/Inputs/Synthetisch/Syn_tot.pkl")
data.head()

,IID,Pheno,Geno,State
0,0pop1,1,"[2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, ...",pop1sick
1,1pop1,1,"[2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, ...",pop1sick
2,2pop1,1,"[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, ...",pop1sick
3,3pop1,0,"[2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",pop1healthy
4,4pop1,1,"[2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2, ...",pop1sick


In [9]:
data.tail()

,IID,Pheno,Geno,State
1995,995pop2,0,"[2, 0, 0, 0, 1, 2, 2, 2, 1, 0, 1, 0, 2, 1, 1, ...",pop2healthy
1996,996pop2,0,"[1, 1, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, ...",pop2healthy
1997,997pop2,0,"[1, 1, 0, 0, 2, 1, 0, 0, 0, 0, 1, 1, 2, 1, 2, ...",pop2healthy
1998,998pop2,0,"[2, 2, 1, 2, 0, 1, 0, 2, 0, 1, 0, 2, 2, 0, 0, ...",pop2healthy
1999,999pop2,0,"[1, 0, 1, 1, 2, 1, 1, 0, 0, 0, 2, 1, 1, 1, 1, ...",pop2healthy


## Preprocessing data into tensors 

### Keeping only the minor alleles as 1

In [74]:
X = only_minor(data["Geno"])
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 0., 1.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

### Train test split

In [75]:
split = splitter([X],0.1)
[X_train, X_test] = tensor_converter(split)
X = tensor_converter(X)
X = tf.stack(X, axis = 0)

## Loading hyper-parameters

In [87]:
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
PRUNING = 0.5
LEARNING_RATE = 0.01
first_layer = 10

In [91]:
input_len = int(tf.shape(X_train)[1])
ff = frequencyfinder(input_len, first_layer)
ff.build((None, input_len))

AttributeError: module 'tensorflow.keras.layers' has no attribute 'IndependentLogistic'

In [82]:
output_test = ff(X)

In [83]:
output_test[0]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.04056555, 0.04056555, 0.1339325 , 0.4581778 , 0.09985406,
       0.04056555, 0.04056555, 0.04056555, 0.06062722, 0.04458066],
      dtype=float32)>

In [95]:
tf.keras.layers.IndependentLogistic()

AttributeError: module 'tensorflow.keras.layers' has no attribute 'IndependentLogistic'

In [97]:
from tensorflow_probability.python import layers as tfpl

ImportError: This version of TensorFlow Probability requires TensorFlow version >= 2.6; Detected an installation of version 2.5.0. Please upgrade TensorFlow to proceed.